In [1]:
import matplotlib.pyplot as plt
import numpy as np
import keras
import tensorflow as tf
import pandas as pd

from google.colab import drive
drive.mount('/mnt/drive') # this loads a folder in my google drive
# you cannot upload files to google colab permanently, so this is workaround

Drive already mounted at /mnt/drive; to attempt to forcibly remount, call drive.mount("/mnt/drive", force_remount=True).


In [ ]:
# !!!!!!!!!!!!!Madhav processing!!!!!!!!:

import os
import numpy as np
import pandas as pd
from PIL import Image, ImageOps

# Define the path to the training data
data_dir = '/mnt/drive/MyDrive/datasets/data/training_data'

# Initialize lists to hold data and labels
data = []
labels = []

label_dirs = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]

# Create a mapping from label to integer (optional, for numerical labels)
label_to_int = {label: idx for idx, label in enumerate(sorted(label_dirs))}

# Loop over each label directory
for label in label_dirs:
    label_dir = os.path.join(data_dir, label)
    print("Currently on label ", label)
    for image_name in os.listdir(label_dir):
        image_path = os.path.join(label_dir, image_name)
        if image_name.lower().endswith(('.png')):
            try:
                # Open the image and convert to grayscale
                img = Image.open(image_path).convert('L')
                x, y = img.size
                if x != y:
                    # Calculate padding amounts
                    delta_w = max(x, y) - x
                    delta_h = max(x, y) - y
                    padding = (delta_w // 2, delta_h // 2, delta_w - (delta_w // 2), delta_h - (delta_h // 2))
                    # Pad with white pixels (255)
                    img = ImageOps.expand(img, padding, fill=255)
                # Resize the image to 28x28 pixels
                img = img.resize((28, 28))
                img_array = np.array(img).flatten()
                # Get the label as integer (or keep as string)
                label_int = label_to_int[label]
                # Append to data and labels
                data.append(img_array)
                labels.append(label_int)
            except Exception as e:
                print(f"Error processing {image_path}: {e}")

# Convert data and labels to pandas DataFrame
df = pd.DataFrame(data)
df.insert(0, 'label', labels)

# Save the DataFrame to a CSV file
# df.to_csv('processed_data.csv', index=False)
df.to_csv('/mnt/drive/MyDrive/datasets/processed_training_data.csv', index=False)

# Approximate 6 minutes ~

In [2]:
# Image preprocessing
train_directory = "/mnt/drive/MyDrive/datasets/processed_training_data.csv"
# A = "/mnt/drive/MyDrive/datasets/data/training_data/A"

# abandoned: "/mnt/drive/MyDrive/datasets/emnist-letters-train.csv"

In [3]:
# Data Import
from sklearn.model_selection import train_test_split
train = pd.read_csv(train_directory)  # this reads my dataset
train.head() # prints the first rows of dataset
train.iloc[:,0] # iloc functions slices the data
# [;,0] is all the y values, labels
# [:,1:] is all the pixel values, 28*28 = 784 columns of pixels
x_train,x_val,y_train,y_val = train_test_split(train.iloc[1:,1:],train.iloc[1:,0],test_size=0.0005,random_state=42)
# test_size is the size of the validation size relative to the whole dataset (here I use validation instead of test)
# this splits data to training sets and validating sets(right now this is not used).

In [4]:
# Data processing
x_train = x_train.values.reshape(x_train.shape[0], 28, 28, 1)
x_val = x_val.values.reshape(x_val.shape[0], 28, 28, 1)
# ensure the matrixes are [28][28][1]
print(y_train.shape)
y_train = keras.utils.to_categorical(y_train - 1, num_classes = 35)
print(y_train.shape)
# get_dummies do this, if your array is 1234, it returns 1 true 2 false 3 false 4 false for the first data, and the rest..
y_val = keras.utils.to_categorical(y_val - 1, num_classes = 35)
print(y_train.shape)

(15992,)
(15992, 35)
(15992, 35)


In [5]:
# Model Building
model = keras.models.Sequential([

    keras.layers.ZeroPadding2D(padding=(2, 2), input_shape=(28,28,1)),
    keras.layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(32,32,1)),
    keras.layers.AveragePooling2D(pool_size=(2, 2)),

    keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'),
    keras.layers.AveragePooling2D(pool_size=(2, 2)),

    keras.layers.Flatten(),

    keras.layers.Dense(units=120, activation='relu'),
    keras.layers.Dense(units=84, activation='relu'),
    keras.layers.Dense(units=35, activation = 'softmax'),
])

# LeNet5 Model (can be remoduled)
# However, input and out sizes must be the same

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/zero_padding2d.py:72: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Model compilation
model.build() # builds the model
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
# you can adjust any of the parameters, or even add metrics
model.fit(x=x_train, y=y_train, epochs=15)
# training the model, realisticly you would want to use more epoches than 5, for here I use fewer epoches for testing

Epoch 1/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.6021 - loss: 2.3703
Epoch 2/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - accuracy: 0.9493 - loss: 0.1989
Epoch 3/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.9673 - loss: 0.1164
Epoch 4/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.9769 - loss: 0.0792
Epoch 5/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.9816 - loss: 0.0616
Epoch 6/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.9859 - loss: 0.0446
Epoch 7/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accuracy: 0.9848 - loss: 0.0437
Epoch 8/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.9903 - loss: 0.0301
Epoch 9/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.9889 - loss: 0.0365
Epoch 10/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accuracy: 0.9922 - loss: 0.0228
Epoch 11/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.9886 - loss: 0.0371
Epoch 12/15
500/500 ━━━━━━━━━━━━━━━

In [7]:
# Predicting the outcome
x_test = pd.read_csv(train_directory) # I will change this later
# x_test = pd.read_csv("/mnt/drive/MyDrive/datasets/emnist-letters-test.csv")
x_test.head()
# read test dataset, or use your own images, anything

x_test = x_test.iloc[:,1:]
x_test = x_test.values.reshape(x_test.shape[0], 28, 28, 1)
y_test = model.predict(x_test) # predicts y value
y_test = np.argmax(y_test, axis=1) # after this line of code, you should get a character value
y_test += 1
print(y_test.max())
print(y_test.min())

501/501 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step
35
1


In [8]:
# Saving the outcome
# y_test = np.array([chr(x + 97) for x in y_test]) # this maps integers to characters
# out = pd.read_csv('/mnt/drive/MyDrive/datasets/out.csv')
out = pd.DataFrame()
out['Label'] = y_test
out.to_csv('/mnt/drive/MyDrive/datasets/processed_test_data_out.csv', index=False)


In [9]:
# Performance Evaluation
test = pd.read_csv(train_directory)
test = test.iloc[:,0].values
#test = np.array([chr(int(x) + 97) for x in test])

matching_values = np.sum(test == y_test)
total_values = y_test.size

# Calculate the percentage
percentage = (matching_values / total_values) * 100
print(percentage)

99.69378827646544
